In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# 라이브러리 불러오기

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib

import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import load_model

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# 사물의 클래스 분류

In [ ]:
basePath = '/content/drive/MyDrive/Colab Notebooks/dacon/'

In [ ]:
test_dir = basePath + "test"
test_dir = pathlib.Path(test_dir)

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
image_count = len(list(test_dir.glob('*/*.png')))
print(image_count)

2154


In [ ]:
# class_names = ['bottle', 'cable', 'capsule', 'carpet', 'grid', 
#                'hazelnut', 'leather', 'metal_nut', 'pill', 'screw', 
#                'tile', 'toothbrush', 'transistor', 'wood', 'zipper']
class_names = os.listdir(basePath + "train/data_train_class")
class_names.sort()
class_names

['bottle',
 'cable',
 'capsule',
 'carpet',
 'grid',
 'hazelnut',
 'leather',
 'metal_nut',
 'pill',
 'screw',
 'tile',
 'toothbrush',
 'transistor',
 'wood',
 'zipper']

In [ ]:
imageList = list(test_dir.glob('*/*.png'))
# print(imageList)

In [ ]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels = None,
    seed = 12,
    shuffle = False,
    image_size = (img_height, img_width),
    batch_size = batch_size
)

Found 2154 files belonging to 1 classes.


In [ ]:
test_df = pd.read_csv(basePath + 'test_df.csv')
test_df.head()

,index,file_name
0,0,20000.png
1,1,20001.png
2,2,20002.png
3,3,20003.png
4,4,20004.png


In [ ]:
model = load_model(basePath + 'model/model_class15.h5')

In [ ]:
img1 = tf.keras.preprocessing.image.load_img(
    '/content/drive/MyDrive/Colab Notebooks/dacon/test/test/20035.png', 
    target_size = (224, 224)
)
img_array1 = tf.keras.preprocessing.image.img_to_array(img1)
img_array1 = tf.expand_dims(img_array1, 0) # Create a batch

predictions = model.predict(img_array1)

print(predictions)

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(predictions[0])], 100 * np.max(predictions[0]))
)

[[0.00269656 0.00281671 0.00124953 0.08187942 0.3104593  0.00524664
  0.03879276 0.01556936 0.02576646 0.00549084 0.0249738  0.01025118
  0.01267008 0.00346517 0.45867208]]
This image most likely belongs to zipper with a 45.87 percent confidence.


In [ ]:
test_list = model.predict(test_ds)

In [ ]:
accuracy_list = []

for i in range(len(test_list)):
    id = np.argmax(test_list[i])
    accuracy_list.append(id)

In [ ]:
accuracy_list2 = accuracy_list.copy()

accuracy_list3 = []
for i in range(len(accuracy_list2)):
  accuracy_list3.append(class_names[accuracy_list[i]])

In [ ]:
accuracy_df = pd.DataFrame(accuracy_list3, columns = ['class']) # 데이터프레임으로 변환
class_df = pd.concat([test_df, accuracy_df], axis = 1)

In [ ]:
class_df.head()

,index,file_name,class
0,0,20000.png,tile
1,1,20001.png,grid
2,2,20002.png,transistor
3,3,20003.png,tile
4,4,20004.png,tile


# 테스트 데이터셋 구축

In [ ]:
from PIL import Image

In [ ]:
filepath = basePath + "test/test"  

classes = os.listdir(filepath)

# 이미지 저장할 상위 폴더
dir_path = basePath + 'class'

# dir_path 폴더 없으면 생성
if os.path.isdir(dir_path) == False:
    os.mkdir(dir_path)

for i, cls in enumerate(classes):
    image = Image.open(filepath + '/' + cls)
    
    filter1 = (class_df['file_name'] == cls)
    df = class_df[filter1]
    class1 = df['class'].values[0]

    # 클래스별 폴더 경로
    folder_path = dir_path + '/' + class1

    # 폴더 없으면 생성
    if os.path.isdir(folder_path) == False:
        os.mkdir(folder_path)

    # 이미지 폴더에 저장
    image.save(folder_path + '/' + cls, format = None)

# 클래스별 상태 분류

In [ ]:
# 사물의 종류 상태 가져오기
def get_state(class1):
    state_path = basePath + 'train/[원본]data_train_state/' + class1
    classes = os.listdir(state_path)
    classes = natsort.natsorted(classes)

    state = []
    for i, cls in enumerate(classes):
        state.append(cls)
        
    print(class1, " : ", state)
    return classes

In [ ]:
import natsort
from PIL import Image

dataPath = basePath + "class" # 클래스별 이미지 경로
classes1 = os.listdir(dataPath)
classes1 = natsort.natsorted(classes1) # 정렬

class_pred_df = pd.DataFrame()
for i, cls in enumerate(classes1):
  print(cls)
  state_dir = basePath + 'class/' + cls
  state_dir = pathlib.Path(state_dir)

  test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    state_dir,
    labels = None,
    seed = 12,
    shuffle = False,
    image_size = (img_height, img_width),
    batch_size = batch_size
  )

  # 클래스별 모델 불러오기
  model_state = load_model(basePath + 'train/model/' + cls + '.h5')
  predictions = model_state.predict(test_ds)

  # 상태 불러오기
  state = get_state(cls)

  state_list = []
  for j in range(len(predictions)):
    if cls == 'toothbrush':
      id = int(np.round(predictions[j]))
    else:
      id = np.argmax(predictions[j])
    state1 = state[id]
    label1 = cls + '-' + state1
    state_list.append(label1)

  # 이미지 파일명 가져오기
  fileNameList = os.listdir(state_dir)
  fileNameList.sort()

  data = pd.DataFrame({'file_name' : fileNameList,
                       'label' : state_list})
  
  class_pred_df = pd.concat([class_pred_df, data], ignore_index = True)

bottle
Found 102 files belonging to 1 classes.
bottle  :  ['broken_large', 'broken_small', 'contamination', 'good']
cable
Found 206 files belonging to 1 classes.
cable  :  ['bent_wire', 'cable_swap', 'combined', 'cut_inner_insulation', 'cut_outer_insulation', 'good', 'missing_cable', 'missing_wire', 'poke_insulation']
capsule
Found 152 files belonging to 1 classes.
capsule  :  ['crack', 'faulty_imprint', 'good', 'poke', 'scratch', 'squeeze']
carpet
Found 136 files belonging to 1 classes.
carpet  :  ['color', 'cut', 'good', 'hole', 'metal_contamination', 'thread']
grid
Found 108 files belonging to 1 classes.
grid  :  ['bent', 'broken', 'glue', 'good', 'metal_contamination', 'thread']
hazelnut
Found 148 files belonging to 1 classes.
hazelnut  :  ['crack', 'cut', 'good', 'hole', 'print']
leather
Found 139 files belonging to 1 classes.
leather  :  ['color', 'cut', 'fold', 'glue', 'good', 'poke']
metal_nut
Found 134 files belonging to 1 classes.
metal_nut  :  ['bent', 'color', 'flip', 'good

In [ ]:
class_pred_df = class_pred_df.sort_values('file_name')
class_pred_df.reset_index(drop = True)

,file_name,label
0,20000.png,tile-glue_strip
1,20001.png,grid-thread
2,20002.png,transistor-good
3,20003.png,tile-glue_strip
4,20004.png,tile-good
...,...,...
2149,22149.png,tile-gray_stroke
2150,22150.png,screw-good
2151,22151.png,grid-good
2152,22152.png,cable-good


In [ ]:
# class_pred_df = pd.DataFrame(class_pred_list, columns = ['file_name', 'label'])
df_test = pd.merge(class_df, class_pred_df, on = 'file_name')
df_test

,index,file_name,class,label
0,0,20000.png,tile,tile-glue_strip
1,1,20001.png,grid,grid-thread
2,2,20002.png,transistor,transistor-good
3,3,20003.png,tile,tile-glue_strip
4,4,20004.png,tile,tile-good
...,...,...,...,...
2149,2149,22149.png,tile,tile-gray_stroke
2150,2150,22150.png,screw,screw-good
2151,2151,22151.png,grid,grid-good
2152,2152,22152.png,cable,cable-good


In [ ]:
df_test = df_test[['index', 'label']]
df_test

,index,label
0,0,tile-glue_strip
1,1,grid-thread
2,2,transistor-good
3,3,tile-glue_strip
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


# 엑셀파일로 저장

In [ ]:
df_test.to_csv(basePath + "test_220427_1.csv", index = False, mode = 'w')